In [141]:
# -*- coding: utf-8 -*-
"""
Annotations reader
Raphaël Ceré
Python 3.6.5
"""

__author__ = "Raphael Cere"
__copyright__ = "Copyright 2019, University of Lausanne, Switzerland"
__credits__ = ["Raphael Cere"]
__license__ = "GNU GPLv3"
__version__ = "0.1a0"
__maintainer__ = "Raphael Cere"
__email__ = "Raphael.Cere@unil.ch"
__status__ = "Development"


def get_data(path):
	"""
	retrun path of data
	"""
	return os.path.dirname(os.path.abspath(path))+"/ANNOTATION_ANT/"

def ann_reader(file):
	"""
	return d : a list of dictionnaries with the annotations per drawing and remove duplicate.
	return h : headers.
	"""
	try:
		with open(file, 'r') as csv_file:
			d = list()
			r_reader = csv.reader(csv_file, delimiter=',')
			h = r_reader.__next__()
			for r in r_reader:
				o = dict()
				[o.update({h[x] : r[x]}) for x in range(0, len(r))]
				d.append(o)
			d = [dict(tupleized) for tupleized in set(tuple(item.items()) for item in d)]
		return d, h
	except IOError: raise IOError("Couldn't read file %s" % file)

def pb_inter(bp0, bp1, head, padding1=0., padding2=0.):
	"""
	retrun True if annotations are overlaping between them (box or point geometry).
	arg bp0 and bp1 : geometry 1 and 2 (point of box)
	arg bp1 : keys dictionnary calling the coordinates
	arg padding1 and padding2 : add a padding in pixel respectivly to geometry 1 and 2
	"""
	pl1, pl2, al = [head[8], head[9]], [head[10], head[11]], head[3]
	if bp0[al] == "point" and bp1[al] == "box":
		x0, y0 = (float(bp0[pl1[0]]), float(bp0[pl1[1]]))
		x1, y1, x2, y2 = (float(bp1[pl1[0]]), float(bp1[pl1[1]]), float(bp1[pl2[0]]),float(bp1[pl2[1]]))
		return True if (x1-padding1 <= x0 <= x2+padding1) and (y1-padding1 <= y0 <= y2+padding1) else False
	if bp1[al] == "point" and bp0[al] == "box":
		x0, y0 = (float(bp1[pl1[0]]), float(bp1[pl1[1]]))
		x1, y1, x2, y2 = (float(bp0[pl1[0]]), float(bp0[pl1[1]]), float(bp0[pl2[0]]),float(bp0[pl2[1]]))
		return True if (x1-padding1 <= x0 <= x2+padding1) and (y1-padding1 <= y0 <= y2+padding1) else False
	elif bp0[al] == "box" and bp1[al] == "box":
		x01, y01, x02, y02 = (float(bp0[pl1[0]]), float(bp0[pl1[1]]), float(bp0[pl2[0]]),float(bp0[pl2[1]]))
		x11, y11, x22, y22 = (float(bp1[pl1[0]]), float(bp1[pl1[1]]), float(bp1[pl2[0]]),float(bp1[pl2[0]]))
		down = True if (x01-padding1 <= x11-padding2 <= x02+padding1) and (y01-padding1 <= y11-padding2 <= y02+padding1) else False
		up = True if (x01-padding1 <= x22+padding2 <= x02+padding1) and (y01-padding1 <= y22+padding2 <= y02+padding1) else False
		return True if down or up else False

def get_annotation(file, inter=False, C1=True, Cx=False):
	"""
	return d : a list of conditonal annotations belonging at R1 per drawing.
	arg inter : call pb_inter()
	C1 or Cx : boolean conditions
	"""
	#print(file)
	if inter:
		b = []
		a, h = ann_reader(file)
		is_r1 = [x for x in a if x[h[1]] == "R1"] # plusieur entres similaire !!!
		if len(is_r1) > 1:
			print("-------------- more than 1 R1: "+file) # file name if more than one R1
		if is_r1: # If intersection R1 True
			if C1 and not Cx: # If R1 is C1
				b = list()
				for r in is_r1: # For each R1
					c_1 = list()
					for c in a: # R1 in C1
						if pb_inter(r, c, h) and c[h[1]] != "R1" and c[h[1]] == "C1":
							c_1.append(c)
					c_in_r1 = list()
					for c in c_1: # X in C1 in R1 not in C
							for x in a:
								if pb_inter(c, x, h) and x[h[1]] != "R1" and x[h[1]][0] != "C":
									c_in_r1.append(x[h[1]])
					b.append(c_in_r1)
			elif Cx and not C1: # If R1 is C
				b = list()
				for r in is_r1: # For each R1
					c_1 = list()
					for c in a: # R1 in C
						if pb_inter(r, c, h) and c[h[1]] != "R1" and c[h[1]][0] == "C":
							c_1.append(c)
					c_in_r1 = list()
					for c in c_1: # X in C1 in R1 not in C
							for x in a:
								if pb_inter(c, x, h) and x[h[1]] != "R1" and x[h[1]][0] != "C":
									c_in_r1.append(x[h[1]])
					b.append(c_in_r1)
			elif not C1 and not Cx: # If R1 is not C
				b = list()
				for r in is_r1: # For each R1
					c_1 = list()
					for c in a: # R1 in C
						if pb_inter(r, c, h) and c[h[1]] != "R1" and c[h[1]][0] != "C":
							c_1.append(c)
					c_in_r1 = list()
					for c in c_1: # X in C1 in R1 not in C
							for x in a:
								if pb_inter(c, x, h) and x[h[1]] != "R1" and x[h[1]][0] != "C":
									c_in_r1.append(x[h[1]])
					b.append(c_in_r1)
			else: # If R1
				b = list()
				for r in is_r1: # For each R1
					c_1 = list()
					for c in a: # R1 in C
						if pb_inter(r, c, h) and c[h[1]] != "R1":
							c_1.append(c)
					c_in_r1 = list()
					for c in c_1: # X in C1 in R1 not in C
							for x in a:
								if pb_inter(c, x, h) and x[h[1]] != "R1":
									c_in_r1.append(x[h[1]])
					b.append(c_in_r1)
		#else: b = []
	else: b, h = ann_reader(file)
	return b, h, file

def filter_annotation(name, drawing,  filter_choice):
	"""
	return annotation : path with the name of the drawing
	return  drawing_filtred : annotations filtred by  filter_choice
	arg filter_choice : list of annotations keys
	comment : it allows to agregate sub keys    
	"""
	#print(drawing)
	drawing_filtred = list()
	if drawing[0]:
		for r in drawing[0]:
			r1_filtred = list()
			for i in filter_choice:
				tmp = list(filter(lambda x: i in x, r ))
				for p in [i]*len(tmp):
					r1_filtred.append(p)
			drawing_filtred.append(r1_filtred)
	else: drawing_filtred = drawing[0]
	return name, drawing_filtred


if __name__ == '__main__':
	import os
	import csv
	import json
	import numpy as np
	import itertools
	import networkx as nx
	from networkx.drawing.nx_agraph import graphviz_layout
	import matplotlib.pyplot as plt
	from itertools import compress

	os.system('cls' if os.name == 'nt' else 'clear')
    
	%matplotlib notebook
	%matplotlib notebook

	# user paramter
	user_path =  "/Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT"
	#user_path = str(input("PATH OF ANNOTATION:"))

In [142]:
	# Read annotations for each drawing in folder
	path = get_data(user_path)
	annotation_drawings = list() 
	for name_file in [x for x in os.listdir(path) if x[:10]=="annotation"]:
		drawing = get_annotation(path+name_file, True, False, True)
		#print(drawing) 
		annotation, drawing_filtred = filter_annotation(name_file, drawing, ["A1.2.2", "A1.2.1","A1.2.2","A1.2.3","A1.2.4","A1.2.5","A1.2.6","A1.2.7","A1.3","A1.4","A1.5","A1.6","A1.7"])
		#print(annotation)
		#print(drawing_filtred)
		annotation_drawings.append((annotation, drawing_filtred))

	#print(annotation_drawings)
	# Counts
	god_all = 0
	draw_with_r1_cond = 0
	for d in annotation_drawings:
		if len(d[1]) >=1:
			god_all += len(d[1])
			draw_with_r1_cond += 1

  
	print(">>> Number of drawing with at least one gods (R1 with C conditions): %s" %(draw_with_r1_cond))
	print(">>> Number of gods annotated (R1 with C conditions): %s" %(god_all))

-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT/annotation_ch10_ge_m_rbc_13_01_pat-r_1280_890_csv_export_karlen_carine
-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT/annotation_jp04_ko_m_ryx_13_xx_nkx-r_678_960_csv_export_sudan_marie-rachel
-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT/annotation_ch10_ge_f_pco_13_04_ann-r_679_960_csv_export_karlen_carine
-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT/annotation_ch10_ge_f_rbc_12_11_jul-r_694_960_csv_export_karlen_carine
-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT/annotation_ch15_fr_f_rbe_09_01_ail-r_680_960_csv_export_dessart_gregory
-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_cvmm/DATA/ANNOTATION_ANT/annotation_ch16_fr_f_rec_13_10_sar-r_678_960_csv_export_dessart_gregory
-------------- more than 1 R1: /Users/rcere/switchdrive/ddd_

In [143]:
	# unique items present ########
	items_by_gods = list()
	for d in annotation_drawings:
		for d_ in d[1]:
			for c in d_:
				items_by_gods.append(c)

	items_unique = list(set(items_by_gods)) # headers
	h = len(items_unique)

	print(">>> %s unique caracteristics." %(h))
	print(items_unique)

>>> 12 unique caracteristics.
['A1.2.3', 'A1.2.6', 'A1.3', 'A1.2.2', 'A1.2.5', 'A1.6', 'A1.5', 'A1.7', 'A1.2.4', 'A1.2.7', 'A1.4', 'A1.2.1']


In [144]:
	# gods x items ########
	drawing_by_god = []
	count_c = []
	for d in annotation_drawings:
		if len(d[1]) >=1:
			for d_ in d[1]:
				drawing_by_god.append(d[0])
				for i in items_unique:
					count_c.append(d_.count(i))

	count_items_by_god = np.array(count_c)[:,np.newaxis].reshape((len(drawing_by_god), len(items_unique)))

	rows = np.array(drawing_by_god, dtype='|S'+str(np.max([len(x) for x in drawing_by_god])))[:, np.newaxis]
	data = np.char.mod("%10.0f", count_items_by_god)
	with open('god_x_items_count.csv', 'w') as f:
		np.savetxt(f, np.hstack((rows, data)), delimiter=', ', fmt='%s', header=str(items_unique))
    
	print(np.shape(data))
	print("Example output: ")
	print(np.hstack((rows, data))[np.random.randint(god_all),:])

(730, 12)
Example output: 
['annotation_ch10_ge_f_rbc_14_10_aur-r_1280_906_csv_export_karlen_carine'
 '         0' '         0' '         0' '         0' '         0'
 '         0' '         0' '         0' '         0' '         0'
 '         0' '         0']


In [145]:
	binary_items_by_god = np.zeros(np.shape(count_items_by_god))
	binary_items_by_god[np.where(count_items_by_god >=1.)] = 1.
	rows = np.array(drawing_by_god , dtype='|S'+str(np.max([len(x) for x in drawing_by_god])))[:, np.newaxis]
	data = np.char.mod("%10.0f", binary_items_by_god)
	with open('god_x_items_binary.csv', 'w') as f:
		np.savetxt(f, np.hstack((rows, data)), delimiter=', ', fmt='%s', header=str(items_unique))

	print(np.shape(data))
	print("Example output: ")
	print(np.hstack((rows, data))[np.random.randint(god_all),:])

(730, 12)
Example output: 
['annotation_ch16_vd_f_rcb_14_05_man-r_677_960_csv_export_christine_rodier'
 '         1' '         1' '         0' '         1' '         1'
 '         0' '         0' '         0' '         1' '         1'
 '         0' '         1']


In [146]:
	# Ouput 1 expected 
	unq, cnt = np.unique(binary_items_by_god, axis=0, return_counts=True)
	with open('god_x_items_unique.csv', 'w') as f:
		np.savetxt(f, np.hstack((unq, cnt[:, np.newaxis])), delimiter=', ', fmt='%s', header=str(items_unique))

	print(np.shape(unq))
	print("Example output: ")
	print(np.hstack((unq, cnt[:, np.newaxis])))

(186, 12)
Example output: 
[[  0.   0.   0. ...   0.   0. 118.]
 [  0.   0.   0. ...   1.   0.   1.]
 [  0.   0.   0. ...   1.   1.   3.]
 ...
 [  1.   1.   1. ...   0.   1.   1.]
 [  1.   1.   1. ...   1.   1.  30.]
 [  1.   1.   1. ...   1.   1.  12.]]


In [149]:
	# All combinaison possible of items ########

	all_comb = list()
	for i in range(np.shape(unq)[0]):
		all_comb.append(list(compress(items_unique, unq[i,:] > 0)))
		#print(items_unique[list(binary_by_items[i,:] > 0)])

	n_comb = len(all_comb)
	net_comb = np.zeros((n_comb, n_comb))
	for i in range(n_comb):
		for j in range(len(all_comb)):
				if len(set(all_comb[i]).intersection(all_comb[j])) == len(all_comb[i]):
					net_comb[i,j] = net_comb[j,i] = 1.
				else: 
					net_comb[i,j] = net_comb[j,i] = net_comb[i,j]

	labels_list = ['|'.join(map(str, x)) for x in all_comb]

	rows = np.array(labels_list, dtype='|S'+str(np.max([len(x) for x in labels_list])))[:, np.newaxis]
	data = np.char.mod("%10.0f", net_comb)
	with open('uitemes_x_uitemes.csv', 'w') as f:
		np.savetxt(f, np.hstack((rows, data)), delimiter=', ', fmt='%s')

In [ ]:
    G.clear()
    G = nx.DiGraph()
    G = nx.Graph()
   
    keep_last = 0
    for j in range(n_comb):
        for i in range(n_comb):
            if net_comb[j,i] == 1.:
                G.add_node(('\n '.join(map(str, all_comb[i])))+'_'+str(len(all_comb[i])))
                G.add_node(('\n '.join(map(str, all_comb[j])))+'_'+str(len(all_comb[j])))
                G.add_edge(('\n '.join(map(str, all_comb[i])))+'_'+str(len(all_comb[i])), ('\n '.join(map(str, all_comb[j])))+'_'+str(len(all_comb[j])))
                keep_last = i
    
    #print(G.edges)
    #pos = nx.spring_layout(G,scale=2)
    pos = nx.nx_pydot.graphviz_layout(G, prog="dot")
    fig2 = nx.draw_networkx(G, pos, arrows=False, with_labels=True, font_size=2,font_family='sans-serif', node_size=300, node_color='white', linewidths=0.25, dpi=1000, width=0.25)
    plt.show()   
    #path_out = str(input("PATH OF OUTPUT:"))
    plt.savefig('network2.png', bbox_inches='tight', pad_inches=0, dpi=300)
    #assert os.path.isfile(path_out+'network2.png'),"Figure note saved. Check the given path."
